In [ ]:
# Data download and unzipping
%%capture
!wget https://raw.githubusercontent.com/AakashSudhakar/2018-data-science-bowl/master/compressed_files/stage1_test.zip -c
!wget https://raw.githubusercontent.com/AakashSudhakar/2018-data-science-bowl/master/compressed_files/stage1_train.zip -c

!mkdir stage1_train stage1_test

!unzip stage1_train.zip -d stage1_train/
!unzip stage1_test.zip -d stage1_test/

In [ ]:
from google.colab import files
uploaded = files.upload() # upload .py files

In [ ]:
%%capture
!pip install keras --upgrade

In [ ]:
import os
import random
import sys
import warnings
import numpy as np
import pandas as pd
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from keras.models import Model, load_model
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore', category=UserWarning, module='skimage')

# Setting seed for reproducability
SEED = 42
np.random.seed(SEED)
%matplotlib inline

In [ ]:
from data_utils import read_train_data, read_test_data, rle_encoding, mask_to_rle
from model import get_unet, iou_metric, iou_metric_batch, my_iou_metric

In [ ]:
# get train_data
X_train,Y_train = read_train_data()
# get test_data
test_img,test_img_sizes = read_test_data()

In [ ]:
#@title Hyperparameter { run: "auto", display-mode: "both" }
BATCH_SIZE = 16 #@param {type:"raw"}
EPOCHS = 50 #@param {type:"raw"}
LEARNING_RATE = 1e-3 #@param {type:"raw"}
OPTIMIZER = optimizers.Adam(lr=LEARNING_RATE) #@param {type:"raw"}
val_size = 0.3 #@param {type:"raw"}

In [ ]:
# train/val split
train_img, val_img, train_mask, val_mask = train_test_split(X_train, Y_train, test_size=val_size, random_state=SEED)
train_size = len(train_img)
val_size = len(val_img)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
data_gen_arg = dict(shear_range=0.5, 
                    rotation_range=50, 
                    zoom_range=0.2, 
                    width_shift_range=0.2, 
                    height_shift_range=0.2,
                    fill_mode='reflect')


img_datagen = ImageDataGenerator(**data_gen_arg)
mask_datagen = ImageDataGenerator(**data_gen_arg)
img_datagen.fit(train_img, augment=True, seed=SEED)
mask_datagen.fit(train_mask, augment=True, seed=SEED)

x = img_datagen.flow(train_img, batch_size=BATCH_SIZE, shuffle=True, seed=SEED)
y = mask_datagen.flow(train_mask, batch_size=BATCH_SIZE, shuffle=True, seed=SEED)

img_datagen_val = ImageDataGenerator()
mask_datagen_val = ImageDataGenerator()
img_datagen_val.fit(val_img, augment=True, seed=SEED)
mask_datagen_val.fit(val_mask, augment=True, seed=SEED)

x_val = img_datagen_val.flow(val_img, batch_size=BATCH_SIZE, shuffle=True, seed=SEED)
y_val = img_datagen_val.flow(val_mask, batch_size=BATCH_SIZE, shuffle=True, seed=SEED)

train_generator = zip(x, y)
val_generator = zip(x_val, y_val)

In [ ]:
u_net_aug = get_unet(OPTIMIZER=OPTIMIZER)

In [ ]:
print("Training...\n")
filepath="best_model.h5" # save best model only
earlystopper = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
checkpointer = ModelCheckpoint(filepath, monitor='val_my_iou_metric', 
                               verbose=1, save_best_only=True, mode='max')
results_aug = u_net_aug.fit_generator(train_generator, validation_data=(val_img, val_mask), 
                    steps_per_epoch=100, epochs=EPOCHS, callbacks=[earlystopper, checkpointer])

In [ ]:
print("Training...\n")
filepath="best_model.h5" # save best model only
earlystopper = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
checkpointer = ModelCheckpoint(filepath, monitor='val_my_iou_metric', 
                               verbose=1, save_best_only=True, mode='max')
results_aug = u_net_aug.fit_generator(train_generator, validation_data=val_generator, 
                    steps_per_epoch=100, validation_steps=10,
                   epochs=EPOCHS, callbacks=[earlystopper, checkpointer])